In [27]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 12 22:05:46 2019

@author: rajgu
"""
import pulp
import pandas as pd
import numpy as np
import os

startTime = pd.datetime.now()

os.chdir("D:\My Personal Documents\Learnings\Data Science\Quantam Stride")
import importlib
import Space_optimisation_config
importlib.reload(Space_optimisation_config)

#print(path_in)
# Instantiate our problem class

optimisation_period_list = Space_optimisation_config.optimisation_period
path_in = Space_optimisation_config.path_in
path_out = Space_optimisation_config.path_out
#data = pd.read_excel(path_in +'Data template - SS_clean_temp.xlsx', sheet_name='Sheet1')
data = pd.read_excel(path_in +'test_data.xlsx')
data.columns = [col.upper().strip().replace(' ', '_').replace('/', '_') for col in data.columns]


data = data.set_index('BUILDING_NAME')


data['LEASE_TERMINATION_DATE'] = pd.to_datetime(data['LEASE_TERMINATION_DATE'], format = '%Y-%m-%d')
data['EARLY_TERMINATION_DATE'] = pd.to_datetime(data['EARLY_TERMINATION_DATE'], errors='coerce')

data.loc[data['EARLY_TERMINATION_DATE'].isnull(), 'EARLY_TERMINATION_DATE'] = \
data.loc[data['EARLY_TERMINATION_DATE'].isnull(), 'LEASE_TERMINATION_DATE']

data['EARLY_TERMINATION_DATE'] = data['EARLY_TERMINATION_DATE'].apply(lambda x: pd.datetime.now().date() if x.year == 2099 else x)
data.loc[data['LEASE_TYPE'].isin(['Owned', 'OWNED']), 'EARLY_TERMINATION_DATE' ] = pd.datetime.now().date()

data.loc[~(data['LEASE_TYPE'].isin(['Owned', 'OWNED'])) & data['EARLY_TERMINATION_DATE'].isnull() & \
           data['LEASE_TERMINATION_DATE'].isnull(), 'EARLY_TERMINATION_DATE' ] = pd.datetime.now().date()


data.loc[data['LEASE_TYPE'].isin(['GROSS', 'Gross']), 'OPEX_SQFT' ] = 0

data['TERMINATION_PENALTY_MNTH'] = 3
data.loc[data['LEASE_TYPE'].isin(['Owned', 'OWNED', 'owned']), 'TERMINATION_PENALTY_MNTH' ] = 0

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
data['PENALTY_PERIOD'] = data['EARLY_TERMINATION_DATE'].apply(lambda x: diff_month(x, pd.datetime.now().date()))
data.loc[data['PENALTY_PERIOD'] < 0, 'PENALTY_PERIOD'] = 0

data['PENALTY_PERIOD']  = data['PENALTY_PERIOD']  + data['TERMINATION_PENALTY_MNTH']

period = 60
for i in range(1, period+1):
    data['PERIOD_'+str(i)] = 0


for i in data.index:
    if data.loc[i,'PENALTY_PERIOD'] > 0:
        period_end = data.loc[i,'PENALTY_PERIOD'] if data.loc[i,'PENALTY_PERIOD'] < 61 else 60
        data.loc[i, 'PERIOD_1': 'PERIOD_'+str(period_end) ] = data.loc[i, 'ANNUAL_RENTAL_COST']/12

output = []
output1 = []
for i in range(len(optimisation_period_list)):
    optimisation_period = optimisation_period_list[i]

    building_status = pulp.LpVariable.dicts("building_status",
                                         (i for i in data.index),
                                         cat='Binary')
    
    seats = pulp.LpVariable.dicts("seats",
                                         (i for i in data.index),
                                         lowBound=0,
                                         cat='Integer')
    
    lease_breakup_status = pulp.LpVariable.dicts("lease_breakup_status",
                                         (i for i in data.index),
                                         cat='Binary')
    
    cost_of_move_in = pulp.LpVariable.dicts("cost_of_move_in",
                                         (i for i in data.index),
                                         lowBound=0,
                                         cat='Integer')
    
    cost_of_move_out = pulp.LpVariable.dicts("cost_of_move_out",
                                         (i for i in data.index),
                                         lowBound=0,
                                         cat='Integer')
    
    
    model = pulp.LpProblem("Cost minimising Lease problem", pulp.LpMinimize)
    
    
    model += pulp.lpSum(
        [building_status[build] * (data.loc[build , 'ANNUAL_RENTAL_COST']/12)* \
        optimisation_period for build in data.index]
        + [lease_breakup_status[build] * (data.loc[build , \
        'PERIOD_1':'PERIOD_'+str(optimisation_period)].sum()) for build in data.index]
        + [cost_of_move_in[build] for build in data.index ]
        + [cost_of_move_out[build] for build in data.index]
        + [building_status[build] * (data.loc[build , 'OPEX_SQFT']/12)* \
        optimisation_period for build in data.index]
        )
     
    
    for build in data.index:
        max_seat = data.loc[build, 'SPACE_OCCUPANCY']
        model += seats[build] <= max_seat * building_status[build]
        model += building_status[build] + lease_breakup_status[build] ==1
        model += cost_of_move_in[build] >= (seats[build] - data.loc[build, 'ASSIGNED_OCCUPANCY']) \
                                            * data.loc[build, 'AVERAGE_MOVEIN_COST']
        model += cost_of_move_out[build] >= (data.loc[build, 'ASSIGNED_OCCUPANCY'] - seats[build]) \
                                            * data.loc[build, 'AVERAGE_MOVEOUT_COST']
    
    
    model += pulp.lpSum(
        [seats[build] for build in data.index]) == pulp.lpSum(
        [data.loc[build,'ASSIGNED_OCCUPANCY'] for build in data.index])
    
    
    model.solve()
    pulp.LpStatus[model.status]
    
    
#    output = []
    for build in data.index:
        var_output = {
            'Report Ran Date': pd.datetime.now().date().strftime('%d-%m-%Y'),
            'Optimisation Period': str(optimisation_period) + ' months',
            'Building': build,
            'Building_status': building_status[build].varValue,
            'Current number of People': data.loc[build, 'ASSIGNED_OCCUPANCY'],
            'Proposed number of People': seats[build].varValue,
            'Cost_of_move_in': cost_of_move_in[build].varValue,
            'Cost_of_move_out': cost_of_move_out[build].varValue,
            'First Year Lease Cost': building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'],
            'First Year OPEX Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'],
            'First Year Lease break down Cost': lease_breakup_status[build].varValue * (data.loc[build , \
                                                'PERIOD_1':'PERIOD_12'].sum()),
            'Total First Year Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] +\
                                    building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] + \
                                    lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_1':'PERIOD_12'].sum()) +\
                                    cost_of_move_in[build].varValue + cost_of_move_out[build].varValue,
    
            'Second Year Lease Cost': building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'],
            'Second Year OPEX Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'],
            'Second Year Lease break down Cost': lease_breakup_status[build].varValue * (data.loc[build , \
                                                'PERIOD_13':'PERIOD_24'].sum()),
            'Total Second Year Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] +\
                                    building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] + \
                                    lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_13':'PERIOD_24'].sum()),
    
            'Third Year Lease Cost': building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'],
            'Third Year OPEX Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'],
            'Third Year Lease break down Cost': lease_breakup_status[build].varValue * (data.loc[build , \
                                                'PERIOD_25':'PERIOD_36'].sum()),
            'Total Third Year Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] +\
                                    building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] + \
                                    lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_25':'PERIOD_36'].sum()),
    
            'Fourth Year Lease Cost': building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'],
            'Fourth Year OPEX Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'],
            'Fourth Year Lease break down Cost': lease_breakup_status[build].varValue * (data.loc[build , \
                                                'PERIOD_37':'PERIOD_48'].sum()),
            'Total Fourth Year Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] +\
                                    building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] + \
                                    lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_37':'PERIOD_48'].sum()),
    
            'Fifth Year Lease Cost': building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'],
            'Fifth Year OPEX Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'],
            'Fifth Year Lease break down Cost': lease_breakup_status[build].varValue * (data.loc[build , \
                                                'PERIOD_49':'PERIOD_60'].sum()),
            'Total Fifth Year Cost': building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] +\
                                    building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] + \
                                    lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_49':'PERIOD_60'].sum())
        }
        output.append(var_output)
        
    yr_actual_cost = sum([data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index]) + \
                               sum([data.loc[build , 'OPEX_SQFT'] for build in data.index])
    
    first_yr_proposed_cost =  sum([building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] for build in data.index])+\
                              sum([building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index])+ \
                              sum([lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_1':'PERIOD_12'].sum()) +\
                                    cost_of_move_in[build].varValue + cost_of_move_out[build].varValue for build in data.index])         
                                    
    
    second_yr_proposed_cost =  sum([building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] for build in data.index])+\
                              sum([building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index])+ \
                              sum([lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_13':'PERIOD_24'].sum()) for build in data.index])
                                     
                                    
    third_yr_proposed_cost =  sum([building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] for build in data.index])+\
                              sum([building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index])+ \
                              sum([lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_25':'PERIOD_36'].sum()) for build in data.index])
                                    
    fourth_yr_proposed_cost =  sum([building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] for build in data.index])+\
                              sum([building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index])+ \
                              sum([lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_37':'PERIOD_48'].sum()) for build in data.index])
                                    
    fifth_yr_proposed_cost =  sum([building_status[build].varValue * data.loc[build , 'OPEX_SQFT'] for build in data.index])+\
                              sum([building_status[build].varValue * data.loc[build , 'ANNUAL_RENTAL_COST'] for build in data.index])+ \
                              sum([lease_breakup_status[build].varValue * (data.loc[build , \
                                    'PERIOD_49':'PERIOD_60'].sum()) for build in data.index])
    total_proposed_cost =  first_yr_proposed_cost + second_yr_proposed_cost  + third_yr_proposed_cost + \
             fourth_yr_proposed_cost   + fifth_yr_proposed_cost     
    proposed_cost = [first_yr_proposed_cost , second_yr_proposed_cost  , third_yr_proposed_cost , \
                     fourth_yr_proposed_cost , fifth_yr_proposed_cost, total_proposed_cost  ]    
    current_cost = [yr_actual_cost, yr_actual_cost, yr_actual_cost, yr_actual_cost, yr_actual_cost, yr_actual_cost*5]                 
                                    
    period_yr = ['First Year', 'Second Year', 'Third Year', 'Fourth Year', 'Fifth Year', 'Total']      
    
    for i,p in enumerate(period_yr):                          
        var_output = {
            'Report Ran Date': pd.datetime.now().date().strftime('%d-%m-%Y'),
            'Optimisation Period': str(optimisation_period) + ' months',
            'Year': p,
            'Current Cost': current_cost[i],
            'Proposed Cost': proposed_cost[i],
            'Saving':  current_cost[i] - proposed_cost[i]
        }
        output1.append(var_output)
    
    
        
output_df = pd.DataFrame.from_records(output)
report_columns = ['Report Ran Date','Optimisation Period','Building', 'Building_status',\
                     'Current number of People', 'Proposed number of People',\
                     'Cost_of_move_in', 'Cost_of_move_out', \
                     'First Year Lease Cost', 'First Year OPEX Cost','First Year Lease break down Cost', \
                     'Total First Year Cost',\
                     'Second Year Lease Cost', 'Second Year OPEX Cost','Second Year Lease break down Cost', \
                     'Total Second Year Cost',\
                     'Third Year Lease Cost', 'Third Year OPEX Cost','Third Year Lease break down Cost', \
                     'Total Third Year Cost',\
                     'Fourth Year Lease Cost', 'Fourth Year OPEX Cost','Fourth Year Lease break down Cost', \
                     'Total Fourth Year Cost',\
                     'Fifth Year Lease Cost', 'Fifth Year OPEX Cost','Fifth Year Lease break down Cost', \
                     'Total Fifth Year Cost']

output_df.to_csv(path_out+'Test Data File1_details.csv', columns= report_columns, index=False)
    
output1_df = pd.DataFrame.from_records(output1)
report1_columns = ['Report Ran Date','Optimisation Period','Year', 'Current Cost',\
                     'Proposed Cost', 'Saving']
output1_df.to_csv(path_out+'Test Data File1_summary.csv', columns= report1_columns, index=False)                         

endTime = pd.datetime.now()

In [2]:
print("Data Size " + str(data.shape))
print("Execution Start Time: " + str(startTime))
print("Execution End Time: " + str(endTime))

Data Size (10, 71)
Execution Start Time: 2019-05-09 19:22:49.520031
Execution End Time: 2019-05-09 19:22:50.836657


In [3]:
data.columns

Index(['ANNUAL_RENTAL_COST', 'LEASE_TERMINATION_DATE',
       'EARLY_TERMINATION_DATE', 'LEASE_TYPE', 'OPEX_SQFT', 'SPACE_OCCUPANCY',
       'ASSIGNED_OCCUPANCY', 'AVERAGE_MOVEIN_COST', 'AVERAGE_MOVEOUT_COST',
       'TERMINATION_PENALTY_MNTH', 'PENALTY_PERIOD', 'PERIOD_1', 'PERIOD_2',
       'PERIOD_3', 'PERIOD_4', 'PERIOD_5', 'PERIOD_6', 'PERIOD_7', 'PERIOD_8',
       'PERIOD_9', 'PERIOD_10', 'PERIOD_11', 'PERIOD_12', 'PERIOD_13',
       'PERIOD_14', 'PERIOD_15', 'PERIOD_16', 'PERIOD_17', 'PERIOD_18',
       'PERIOD_19', 'PERIOD_20', 'PERIOD_21', 'PERIOD_22', 'PERIOD_23',
       'PERIOD_24', 'PERIOD_25', 'PERIOD_26', 'PERIOD_27', 'PERIOD_28',
       'PERIOD_29', 'PERIOD_30', 'PERIOD_31', 'PERIOD_32', 'PERIOD_33',
       'PERIOD_34', 'PERIOD_35', 'PERIOD_36', 'PERIOD_37', 'PERIOD_38',
       'PERIOD_39', 'PERIOD_40', 'PERIOD_41', 'PERIOD_42', 'PERIOD_43',
       'PERIOD_44', 'PERIOD_45', 'PERIOD_46', 'PERIOD_47', 'PERIOD_48',
       'PERIOD_49', 'PERIOD_50', 'PERIOD_51', 'PERIOD_52', 

In [37]:
building_status[build]

building_status_A400

In [35]:
building_status[build]

building_status_A400

In [36]:
build

'A400'

In [47]:
data.to_csv("test.csv")